In [1]:
import numpy             as np
import pandas            as pd
import seaborn           as sns
import matplotlib.pyplot as plt

import torch
import torch.nn            as nn
import torch.nn.functional as F

import sklearn

from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics  import accuracy_score

import libs.mol_process as molp

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando dispositivo: {device}')

# coordinate range [-3.368006, 19.782751] (23.150757000000002)
MIN_COOR = -3.368006
MAX_COOR = 19.782751
RANGE    = MAX_COOR - MIN_COOR

MIN_DIFF     = 0.04481336368763274
AVERAGE_DIFF = 0.8968847383141587

used_bins     = 30
used_range    = RANGE + (RANGE/100)
used_min_coor = MIN_COOR

FLATTENED_SIZE = 27

elements = {
	'H' : 1,
	'C' : 6, 'N' : 7, 'O' : 8, 'F' : 9,
	'Si':14, 'P' :15, 'S' :16, 'Cl':17,
	'Ca':20, 
	'Mo':44
}

Usando dispositivo: cuda


In [3]:
dataset = molp.MolDataset(
	'./datasets/test_systems',
	element_dict =elements,
	_range       =used_range,
	n_bins       =used_bins,
	normalize_val=44,
	device       =device
)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

X, y = next(iter(dataloader))
print(y.shape)

# X = molp.MolDataLoader(
# 	'./datasets/test_systems/*.xyz',
# 	element_dict =elements,
# 	shuffle      =True,
# 	offset       =used_min_coor,
# 	_range       =used_range,
# 	n_bins       =used_bins,
# 	normalize_val=44,
# 	device       =device
# )
# y = molp.BandGapDataLoader(
# 	'./datasets/test_systems/*.XV',
# 	device=device
# )

torch.Size([10])


In [4]:
class MoleculeModel(nn.Module):
	def __init__(self, dim_space=30,*args, **kwargs) -> None:
		super().__init__(*args, **kwargs)

		self.conv_l1 = nn.Conv3d(in_channels= 1,out_channels=32,kernel_size=(3,3,3),padding=1)
		self.conv_l2 = nn.Conv3d(in_channels=32,out_channels=32,kernel_size=(3,3,3),padding=1)
		self.pool_1a = nn.AvgPool3d(kernel_size=(3,3,3))

		self.conv_l3 = nn.Conv3d(in_channels=32,out_channels=64,kernel_size=(3,3,3),padding=1)
		self.conv_l4 = nn.Conv3d(in_channels=64,out_channels=1,kernel_size=(3,3,3),padding=1)
		self.pool_2m = nn.MaxPool3d(kernel_size=(3,3,3))

		self.dense_1 = nn.Linear(in_features=FLATTENED_SIZE, out_features=128)
		self.relu_1  = nn.ReLU()
		self.dense_2 = nn.Linear(128, 1)

	def forward(self, x):
		out = self.conv_l1(x)
		out = self.conv_l2(out)
		out = self.pool_1a(out)

		out = self.conv_l3(out)
		out = self.conv_l4(out)
		out = self.pool_2m(out)
		out = out.reshape(out.size(0), -1)

		out = self.dense_1(out)
		out = self.relu_1(out)
		out = self.dense_2(out)

		return out

In [5]:
model = MoleculeModel(used_bins).to(device)
result = model(X[0].view(1, used_bins, used_bins, used_bins))
print(result.shape)
print(result)

torch.Size([1, 1])
tensor([[0.1029]], device='cuda:0', grad_fn=<AddmmBackward0>)
